In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [4]:
def get_sphere(n):
    """get_sphere(n) discretized a sphere using a Fibonacci lattice with midpoint intertion (and poles added by hand)
       input parameters:
       n (int) number of points in the discretization (including poles)
       returns:       
       sphere <class 'scipy.spatial._qhull.ConvexHull'> with points and triangulation according to convex hull
              (check https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html)
       phi (numpy array, len=n) array with phi angles 
       theta (numpy array, len=n) array with theta angles        
       
       more info in "Measurement of Areas on a Sphere Using Fibonacci and Latitude–Longitude Lattices" by
        Alvaro Gonzalez, Math Geosci (2010) 42: 49–64, DOI 10.1007/s11004-009-9257-x. 
        Check also https://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
    """
    import numpy as np
    import scipy.spatial as sp
    #Golden ratio
    gr=(1.+np.sqrt(5.))/2.
    #ng=number of points by Fibonacci lattice with midpoint insertion
    ng=n-2
    i=np.arange(ng,dtype=int)
    phi=np.zeros(n)
    theta=np.zeros(n)
    phi[1:ng+1]=2*np.pi*i/gr
    theta[1:ng+1]=np.arccos(1.-2*(i+0.5)/ng)
    #adding poles by hand
    phi[0]=0.
    theta[0]=0.
    phi[n-1]=0.
    theta[n-1]=np.pi
    #getting Cartesian coordinates
    points=np.zeros((n,3))    
    sin_arr=np.sin(theta)
    points[:,0]=np.cos(phi)*sin_arr
    points[:,1]=np.sin(phi)*sin_arr
    points[:,2]=np.cos(theta)
    #getting convex hull
    sphere=sp.ConvexHull(points)
    
    
    return sphere,phi,theta


In [5]:
sphere,phi,theta=get_sphere(1000)

In [ ]:
lambdas = np.array([8,2.5])
r_proj =  1e6 # radial distance away from the origin at which to project fields
theta_proj=theta
phi_proj=phi
sizes_exp = np.array([5.4,8.4,9.4,10.4,11.4])-1.4
cuts = sizes_exp/(14.3*0.8)

In [7]:
folder_path = rf"./Structures"
project_name = "20251006 Far Field Transmission LSU ff_21p72_5x_size"
runtime_ps = 25e-12
min_steps_per_lambda = 20
ref = False

In [ ]:
ref = False
h5_bg = None
for direction in ["z"]: 
    for f,filename in enumerate(os.listdir(folder_path)):
     for cut in cuts:
        if not (Path(filename).suffix==".h5"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=14.3*0.8,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, multiplicate_size=True,multiplication_factor=5,
                                           monitors=["flux", "far_field"], flux_monitor_position=12.5,cell_size_manual=35,
                                           freqs=250, far_field_settings={"r":r_proj, "theta":theta_proj,"phi":phi_proj, "window_size":0.25},
                                           cut_condition=cut, source="gaussian", gaussian_params={"waist_radius":14,"waist_distance":-16,"theta":22.5*np.pi/180,"phi":0, "position_x":-19, "size":28}, 
                                           absorbers=130, boundaries= "absorbers",
                                            use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg:.2f}" if h5_bg else ""),h5_bg=h5_bg#, ref_only=True,
                                           )
        
        file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value:.2f}\z_incidence\{structure_1.sim_name}.txt"
        if os.path.exists(file_desc):
            print("Exist!")
        else:
           print("Creating...")
           structure_1.run_sim(run_free=False,load=False,add_ref=ref,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value:.2f}",monitor=True)
           if ref:
              ref = False

#         del structure_1


        # structure_1.estimate_cost()
        # structure_1.sim.plot_3d()
        # raise Exception


Configured successfully.


09:18:47 W. Europe Standard Time WARNING: Structure at 'structures[0]' has      
                                 bounds that extend exactly to simulation edges.
                                 This can cause unexpected behavior. If         
                                 intending to extend the structure to infinity  
                                 along one dimension, use td.inf as a size      
                                 variable instead to make this explicit.        

                                 WARNING: Suppressed 19 WARNING messages.       

                                 WARNING: Field projection monitor 'far_field'  
                                 has observation points set up such that the    
                                 monitor is projecting backwards with respect to
                                 its 'normal_dir'. If this was not intentional, 
                                 please take a look at the documentation        
                                 associated with this type of projection monitor
                                 to check how the observation point coordinate  
                                 system is defined.                             

Creating...


...

Output()

10:21:43 W. Europe Standard Time status = preprocess

10:30:11 W. Europe Standard Time starting up solver

                                 running solver

Output()